In [1]:
import BAC0

In [2]:
bacnet = BAC0.connect(port=47811)

2025-01-16 15:27:15,453 - INFO    | Starting BAC0 version 23.07.03 (Lite)
2025-01-16 15:27:15,454 - INFO    | Use BAC0.log_level to adjust verbosity of the app.
2025-01-16 15:27:15,456 - INFO    | Ex. BAC0.log_level('silence') or BAC0.log_level('error')
2025-01-16 15:27:15,458 - INFO    | Starting TaskManager
2025-01-16 15:27:15,468 - WARNING | Netifaces not installed on your system. BAC0 can't detect the subnet.
Please provide subnet for now, we'll consider 255.255.255.0 (/24).
You can install netifaces using 'pip install netifaces'.
2025-01-16 15:27:15,472 - INFO    | Using ip : 10.84.8.66:47811 on port 47811 | broadcast : 10.84.8.255
2025-01-16 15:27:15,536 - INFO    | Starting app...
2025-01-16 15:27:15,556 - INFO    | BAC0 started
2025-01-16 15:27:15,557 - INFO    | Registered as Simple BACnet/IP App
2025-01-16 15:27:15,558 - INFO    | Device instance (id) : 3056204
2025-01-16 15:27:15,589 - INFO    | Update Local COV Task started (required to support COV)


2025-01-16 22:08:59,163 - WARNING | High latency for update_local_cov_Update Local COV Task
2025-01-16 22:09:00,479 - WARNING | Stats : update_local_cov_Update Local COV Task   | Avg exec delay : 8624.47 sec | Avg latency : 8619.47 sec | last executed : 2025-01-16 17:21:34 | Next Time : 2025-01-16 17:21:39
2025-01-16 22:09:00,503 - WARNING | High latency for update_local_cov_Update Local COV Task
2025-01-16 22:09:00,506 - WARNING | Stats : update_local_cov_Update Local COV Task   | Avg exec delay : 8624.78 sec | Avg latency : 8619.78 sec | last executed : 2025-01-16 17:21:35 | Next Time : 2025-01-16 17:21:40
2025-01-16 22:09:00,580 - WARNING | High latency for Ping Task
2025-01-16 22:09:00,590 - WARNING | Stats : Ping Task                                | Avg exec delay : 8901.43 sec | Avg latency : 8601.43 sec | last executed : 2025-01-16 17:17:17 | Next Time : 2025-01-16 17:22:17
2025-01-16 22:09:00,613 - WARNING | High latency for Ping Task
2025-01-16 22:09:00,616 - WARNING | Stats 

In [9]:
def leer_punto(ip,tipo,direccion):
    try:
        return bacnet.read(f'{ip} {tipo} {direccion} presentValue')
    except:
        return ''


In [11]:
leer_punto("10.84.67.185", "binaryOutput", "257")

#bacnet.read(f'10.84.67.185 binaryOutput 257 presentValue')

'active'

In [9]:
# bacnet.write("10.84.67.185 binaryOutput 257 presentValue active"
# bacnet.write(f"10.84.67.185 multiStateOutput 267 presentValue 3")
bacnet.write(f"10.84.67.185 analogValue 269 presentValue 21.0")


In [44]:
tabla = bacnet.readMultiple("10.84.67.185 device 1001 all")

In [45]:
puntos = tabla[-15]
tabla[-15]

[('device', 9001),
 ('binaryOutput', 257),
 ('binaryInput', 258),
 ('binaryOutput', 259),
 ('binaryInput', 260),
 ('binaryInput', 261),
 ('binaryValue', 262),
 ('multiStateOutput', 263),
 ('multiStateInput', 264),
 ('binaryOutput', 265),
 ('binaryInput', 266),
 ('multiStateOutput', 267),
 ('multiStateInput', 268),
 ('analogValue', 269),
 ('analogInput', 270),
 ('binaryInput', 271),
 ('analogInput', 272),
 ('multiStateOutput', 273),
 ('multiStateInput', 274),
 ('analogInput', 275),
 ('analogInput', 276),
 ('multiStateOutput', 277),
 ('multiStateInput', 278),
 ('binaryOutput', 279),
 ('binaryInput', 280),
 ('binaryOutput', 281),
 ('binaryInput', 282),
 ('analogValue', 283),
 ('analogValue', 284),
 ('analogInput', 285),
 ('analogInput', 286),
 ('binaryOutput', 287),
 ('binaryInput', 288),
 ('binaryOutput', 289),
 ('binaryInput', 290),
 ('binaryOutput', 291),
 ('binaryInput', 292),
 ('analogValue', 293),
 ('analogInput', 294),
 ('analogValue', 295),
 ('analogInput', 296),
 ('analogValue', 

In [32]:
import json
mapeo_modbus = []
coil_offset = 0
register_offset = 0

for tipo, direccion in puntos:
    if tipo in ["binaryInput", "binaryOutput"]:
        mapeo_modbus.append({
            "bacnet_tipo": tipo,
            "bacnet_direccion": direccion,
            "modbus_tipo": "Coil",
            "modbus_direccion": coil_offset,
        })
        coil_offset += 1
    elif tipo in ["analogInput", "analogOutput", "analogValue"]:
        mapeo_modbus.append({
            "bacnet_tipo": tipo,
            "bacnet_direccion": direccion,
            "modbus_tipo": "Holding Register",
            "modbus_direccion": register_offset,
        })
        register_offset += 1

with open("archivo.json", "w") as archivo:
    json.dump(mapeo_modbus, archivo, indent=4)

In [6]:
from pymodbus.client import ModbusTcpClient

# Configuración del cliente
client = ModbusTcpClient('10.84.67.18', port=502)

# Conexión
if client.connect():
    # Lectura de 10 registros de retención desde la dirección 0
    response = client.read_holding_registers(address=0, count=10, unit=1)

    if not response.isError():  # Verifica si no hubo errores
        print("Registros:", response.registers)
    else:
        print("Error al leer registros:", response)

    # Cerrar conexión
    client.close()
else:
    print("No se pudo conectar al dispositivo Modbus.")


Connection to (10.84.67.18, 502) failed: timed out


No se pudo conectar al dispositivo Modbus.


In [11]:
import pymodbus
from pymodbus.client import ModbusTcpClient

In [15]:
# Crear una conexión con el dispositivo Modbus
client = pymodbus.client.ModbusTcpClient("10.84.67.185", 502)

# Leer el valor del registro de datos
register = 10
response = client.read_holding_registers(register, 1)

# Imprimir el valor del registro de datos
print(response.registers[0])

AttributeError: module 'pymodbus.client' has no attribute 'ModbusClient'

In [14]:
help(ModbusClient)


Help on class ModbusTcpClient in module pymodbus.client.tcp:

class ModbusTcpClient(pymodbus.client.base.ModbusBaseSyncClient)
 |  ModbusTcpClient(host: 'str', *, framer: 'FramerType' = <FramerType.SOCKET: 'socket'>, port: 'int' = 502, name: 'str' = 'comm', source_address: 'tuple[str, int] | None' = None, reconnect_delay: 'float' = 0.1, reconnect_delay_max: 'float' = 300, timeout: 'float' = 3, retries: 'int' = 3, trace_packet: 'Callable[[bool, bytes], bytes] | None' = None, trace_pdu: 'Callable[[bool, ModbusPDU], ModbusPDU] | None' = None, trace_connect: 'Callable[[bool], None] | None' = None) -> 'None'
 |  
 |  **ModbusTcpClient**.
 |  
 |  Fixed parameters:
 |  
 |  :param host: Host IP address or host name
 |  
 |  Optional parameters:
 |  
 |  :param framer: Framer name, default FramerType.SOCKET
 |  :param port: Port used for communication
 |  :param name: Set communication name, used in logging
 |  :param source_address: source address of client
 |  :param reconnect_delay: Not us

In [47]:
import BAC0
bacnet = BAC0.connect(port=47815)

2025-01-20 15:26:10,073 - INFO    | Starting BAC0 version 23.07.03 (Lite)
2025-01-20 15:26:10,077 - INFO    | Use BAC0.log_level to adjust verbosity of the app.
2025-01-20 15:26:10,078 - INFO    | Ex. BAC0.log_level('silence') or BAC0.log_level('error')
2025-01-20 15:26:10,191 - WARNING | Netifaces not installed on your system. BAC0 can't detect the subnet.
Please provide subnet for now, we'll consider 255.255.255.0 (/24).
You can install netifaces using 'pip install netifaces'.
2025-01-20 15:26:10,202 - INFO    | Using ip : 10.84.8.72:47815 on port 47815 | broadcast : 10.84.8.255


InitializationError: IP Address provided (10.84.8.72:47815) already used by BAC0. Check if another software is using port 47808 on this network interface. If so, you can define multiple IP per interface. Or specify another IP using BAC0.lite(ip='IP/mask')

In [48]:
#bacnet.read(f'10.84.16.195 analogValue 525 presentValue')
tabla = bacnet.readMultiple("10.84.16.195 device 0 all")
#bacnet.readMultiple("10.84.67.185 device 1001 all")
for i in range(50):
    if tabla[i]:
        print(tabla[i])
# Tipo de punto con direcciones print(tabla[13])

('device', 0)
DAIKIN MasterStation III No 0
device
operational
DAIKIN Industries LTD
53
D-BACS BACnet Gateway
Ver.000.001
06_30_00
1
4
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0]
[1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[('device', 0), ('binaryOutput', 1), ('binaryInput', 2), ('binaryInput', 3), ('multiStateInput', 4), ('multiStateOutput', 5), ('multiStateInput', 6), ('multiStateOutput', 7), ('multiStateInput', 8), ('analogInput', 9), ('analogValue', 10), ('binaryInput', 11), ('binaryValue', 12), ('binaryValue', 13), ('binaryValue', 14), ('binaryValue', 16), ('accumulator', 18), ('accumulator', 19), ('binaryInput', 20), ('analogValue', 22), ('analogInput', 23), ('binaryOutput', 24), ('binaryInput', 25), ('binaryOutput', 26), ('binaryInput', 27), ('binaryInput', 28), ('binaryInput', 29), ('binaryInput', 30), ('binaryInput', 31), ('binaryOutput', 257), ('binaryInput', 258), ('b

IndexError: list index out of range

In [ ]:
bacnet.read("10.84.49.185 analogInput 41235 presentValue")
bacnet.read("10.84.49.185 analogInput 41235 presentValue")





21.0

In [1]:
import BAC0
bacnet = BAC0.connect(port=47815)

2025-01-22 11:33:05,906 - INFO    | Starting BAC0 version 23.07.03 (Lite)
2025-01-22 11:33:05,907 - INFO    | Use BAC0.log_level to adjust verbosity of the app.
2025-01-22 11:33:05,907 - INFO    | Ex. BAC0.log_level('silence') or BAC0.log_level('error')
2025-01-22 11:33:05,907 - INFO    | Starting TaskManager
2025-01-22 11:33:06,003 - WARNING | Netifaces not installed on your system. BAC0 can't detect the subnet.
Please provide subnet for now, we'll consider 255.255.255.0 (/24).
You can install netifaces using 'pip install netifaces'.
2025-01-22 11:33:06,005 - INFO    | Using ip : 10.84.8.34:47815 on port 47815 | broadcast : 10.84.8.255
2025-01-22 11:33:09,736 - INFO    | Starting app...
2025-01-22 11:33:09,737 - INFO    | BAC0 started
2025-01-22 11:33:09,739 - INFO    | Registered as Simple BACnet/IP App
2025-01-22 11:33:09,740 - INFO    | Device instance (id) : 3056978
2025-01-22 11:33:09,756 - INFO    | Update Local COV Task started (required to support COV)


2025-01-22 20:40:21,283 - WARNING | High latency for update_local_cov_Update Local COV Task
2025-01-22 20:40:21,610 - WARNING | Stats : update_local_cov_Update Local COV Task   | Avg exec delay : 6256.07 sec | Avg latency : 6251.07 sec | last executed : 2025-01-22 17:11:54 | Next Time : 2025-01-22 17:11:59
2025-01-22 20:40:21,625 - WARNING | High latency for Ping Task
2025-01-22 20:40:21,627 - WARNING | Stats : Ping Task                                | Avg exec delay : 6513.87 sec | Avg latency : 6213.87 sec | last executed : 2025-01-22 17:08:13 | Next Time : 2025-01-22 17:13:13
2025-01-22 20:40:26,614 - WARNING | High latency for update_local_cov_Update Local COV Task
2025-01-22 20:40:26,616 - WARNING | Stats : update_local_cov_Update Local COV Task   | Avg exec delay : 3130.54 sec | Avg latency : 3125.54 sec | last executed : 2025-01-22 20:40:21 | Next Time : 2025-01-22 20:40:26
2025-01-23 18:54:18,516 - WARNING | High latency for update_local_cov_Update Local COV Task
2025-01-23 18

# Daikin

In [16]:

# Leyendo puntos del dispositivo Daikin1 por BACnetIP
lecturas = {
    'StartStopCommand_001':f'10.84.16.195 binaryOutput 257 presentValue', # [ (1) Comando Estado] inactive, active
    'StartStopStatus_001':f'10.84.16.195 binaryInput 258 presentValue', # [ (2) Estado] inactive, active
    'Alarm_001':f'10.84.16.195 binaryInput 259 presentValue',  
    'MalfunctionCode_001':f'10.84.16.195 multiStateInput 260 presentValue', # [ (7) Error] 1: OK
    'AirConModeCommand_001':f'10.84.16.195 multiStateOutput 261 presentValue', 
    'AirConModeStatus_001':f'10.84.16.195 multiStateInput 262 presentValue', 
    'AirFlowRateCommand_001':f'10.84.16.195 multiStateOutput 263 presentValue',# [ (3) Comando Velocidad]
    'AirFlowRateStatus_001':f'10.84.16.195 multiStateInput 264 presentValue', # 1: [ (4) Velocidad] low, 2: high
    'RoomTemp_001':f'10.84.16.195 analogInput 265 presentValue', # [(6) Temperatura]
    'TempAdjust_001':f'10.84.16.195 analogValue 266 presentValue', # [ (5) Setpoint]
    'FilterSign_001':f'10.84.16.195 binaryInput 267 presentValue',
    'FilterSignReset_001':f'10.84.16.195 binaryValue 268 presentValue',
    'RemoteControlStart_001':f'10.84.16.195 binaryValue 269 presentValue',
    'RemoteControlAirConModeSet_001':f'10.84.16.195 binaryValue 270 presentValue',
    'RemoteControlTempAdjust_001':f'10.84.16.195 binaryValue 272 presentValue',
    'GasTotalPower_001':f'10.84.16.195 accumulator 274 presentValue',
    'ElecTotalPower_001':f'10.84.16.195 accumulator 275 presentValue',
    'CommunicationStatus_001':f'10.84.16.195 binaryInput 276 presentValue',
    'AirDirectionCommand_001':f'10.84.16.195 analogValue 278 presentValue',
    'AirDirectionStatus_001':f'10.84.16.195 analogInput 279 presentValue',
    'ForcedThermoOFFCommand_001':f'10.84.16.195 binaryOutput 280 presentValue',
    'ForcedThermoOFFStatus_001':f'10.84.16.195 binaryInput 281 presentValue',
    'EnergyEfficiencyCommand_001':f'10.84.16.195 binaryOutput 282 presentValue',
    'EnergyEfficiencyStatus_001':f'10.84.16.195 binaryInput 283 presentValue',
    'ThermoStatus_001':f'10.84.16.195 binaryInput 284 presentValue',
    'CompressorStatus_001':f'10.84.16.195 binaryInput 285 presentValue',
    'IndoorFanStatus_001':f'10.84.16.195 binaryInput 286 presentValue',
    'HeaterStatus_001':f'10.84.16.195 binaryInput 287 presentValue',
}
for nombre_objeto, lectura in lecturas.items():
    print(f'{nombre_objeto}: {bacnet.read(lectura)} {type(bacnet.read(lectura))}')

StartStopCommand_001: active <class 'str'>
StartStopStatus_001: active <class 'str'>
Alarm_001: inactive <class 'str'>
MalfunctionCode_001: 1 <class 'int'>
AirConModeCommand_001: 1 <class 'int'>
AirConModeStatus_001: 1 <class 'int'>
AirFlowRateCommand_001: 2 <class 'int'>
AirFlowRateStatus_001: 2 <class 'int'>
RoomTemp_001: 23.799999237060547 <class 'float'>
TempAdjust_001: 21.0 <class 'float'>
FilterSign_001: inactive <class 'str'>
FilterSignReset_001: inactive <class 'str'>
RemoteControlStart_001: inactive <class 'str'>
RemoteControlAirConModeSet_001: inactive <class 'str'>
RemoteControlTempAdjust_001: inactive <class 'str'>
GasTotalPower_001: 0 <class 'int'>
ElecTotalPower_001: 0 <class 'int'>
CommunicationStatus_001: inactive <class 'str'>
AirDirectionCommand_001: 0.0 <class 'float'>
AirDirectionStatus_001: 0.0 <class 'float'>
ForcedThermoOFFCommand_001: inactive <class 'str'>
ForcedThermoOFFStatus_001: inactive <class 'str'>
EnergyEfficiencyCommand_001: inactive <class 'str'>
Ener

In [9]:
# Para la escritura se van a usar los registros del 1000 hacia adelante,
# Escribiendo puntos del dispositivo Daikin1 por BACnetIP
escrituras = {
    'Encendido': f'10.84.16.195 binaryOutput 257 presentValue active', # StartStopCommand
    'Velocidad': f'10.84.16.195 multiStateOutput 263 presentValue 2', # AirFlowRateCommand
    'Setpoint': f'10.84.16.195 analogValue 266 presentValue 20.0' # TempAdjust funciona como punto de lectura/escritura
}
bacnet.write(escrituras['Setpoint']) 

#bacnet.write(f"10.84.67.185 analogValue 269 presentValue 21.0")

In [ ]:
# Cliente BACnet
import BAC0

bacnet = BAC0.connect(port=47811)

In [13]:
# Lectura mediante protocolo BACnet de un equipo (gateway) Daikin
# bacnet.read(f'{IP del equipo} {tipo de objeto} {direccion del punto} presentValue')
# bacnet.read(f'10.84.16.195 binaryInput 258 presentValue')
bacnet.read(f'10.84.16.195 analogValue 266 presentValue')

19.0

In [14]:
# Escritura (control). El punto debe ser un punto "Command"
# bacnet.write(f'{IP del equipo} {tipo de objeto} {direccion del punto} presentValue {valor}')
# valor para Estado: {active, inactive}; para Velocidad {1,2,3}; para Setpoint {valor flotante}
#bacnet.write(f'10.84.16.195 binaryOutput 257 presentValue active')
bacnet.write(f'10.84.16.195 analogValue 266 presentValue 21.0')